# Example : deviation of a cantilever beam
## Presentation of the study case
This Example Guide regroups several Use Cases described in the Use Cases Guide in order to show one example of a complete study.

This example has been presented in the ESREL 2007 conference in the paper : OpenTURNS, an Open source initiative to Treat Uncertainties, Risks'N Statistics in a structured industrial approach, from A. Dutfoy(EDF R&D), I. Dutka-Malen(EDF R&D), R. Lebrun (EADS innovation Works) et al.

Let's consider the following analytical example of a cantilever beam, of Young's modulus E, length L, section modulus I. One end is built in a wall and we apply a concentrated bending load at the other end of the beam. The deviation (vertical displacement) y of the free end is equal to :

y(E,F,L,I)=FL 3 3EI

The objective of this study is to evaluate the influence of uncertainties of the input data (E,F,L,I) on the deviation y.

We consider a steel beam with a hollow square section of length a and of thickness e. Thus, the flexion section inertie of the beam is equal to I=a 4 -(a-e) 4 12. The beam length is L. The Young's modulus is E. The charge applied is F.

The values used for the deterministic studies are :

E=3.0e9PaF=300NL=2.5mI=4.0e-6m 4 .
which corresponds to the point (3.0e7,30000,250,400) when the length L is given in unit cm et noo in the standard unit m.

This example treats the Monte Carlo simulation method points of the methodology.

In [6]:
from openturns import *
from openturns.viewer import View
from math import sqrt
#import matplotlib.pyplot as plt
import matplotlib.pyplot 

#######################
### Function 'deviation'
#######################
# Create here the python lines to define the implementation of the function

# In order to be able to use that function with the openturns library,
# it is necessary to define a class which derives from OpenTURNSPythonFunction

class modelePYTHON(OpenTURNSPythonFunction) :
  # that following method defines the input size (4) and the output size (1)
  def __init__(self) :
    OpenTURNSPythonFunction.__init__(self,4,1)

  # that following method gives the implementation of modelePYTHON
  def _exec(self,x) :
    E=x[0]
    F=x[1]
    L=x[2]
    I=x[3]
    return [(F*L*L*L)/(3.*E*I)]

# Use that function defined in the script python
# with the openturns library
# Create a NumericalMathFunction from modelePYTHON
# Create the marginal distriutions of the input random vector
distributionE = Beta(0.93, 3.2, 2.8e7, 4.8e7)
distributionF = LogNormal()
distributionF.setParameter(LogNormalMuSigma()([30000, 9000, 15000]))
#distributionF = LogNormal(30000, 9000, 15000, LogNormalMuSigma)
#distributionF = LogNormal(30000, 9000, 15000)
distributionL = Uniform(250, 260)
distributionI = Beta(2.5, 4.0, 3.1e2, 4.5e2)
# Create a collection of the marginal distributions
collectionMarginals = [distributionE, distributionF, distributionL, distributionI]

# Create the Spearman correlation matrix of the input random vector
RS = CorrelationMatrix(4)
RS[2,3] = -0.2

# Evaluate the correlation matrix of the Normal copula from RS
R = NormalCopula.GetCorrelationFromSpearmanCorrelation(RS)

# Create the Normal copula parametrized by R
copuleNormal = NormalCopula(R)

# Create the input probability distribution of dimension 4
inputDistribution = ComposedDistribution(collectionMarginals, copuleNormal)
# Create the input random vector
inputRandomVector = RandomVector(inputDistribution)
inputRandomVector.setDescription( ("E","F","L","I") )

deviation = NumericalMathFunction(modelePYTHON())
outputVariableOfInterest =  RandomVector(deviation, inputRandomVector)
size = 10000
output_sample = outputVariableOfInterest.getSample(size)

threshold = 30
myEvent = Event(outputVariableOfInterest, Greater(), threshold)
myEvent.setName("Deviation > 30 cm")

print ("############")
print ("Monte Carlo")
print ("############")
print ("")


maximumOuterSampling = 40000
blockSize = 100
coefficientOfVariation = 0.10

# We create a Monte Carlo algorithm
myAlgoMonteCarlo = MonteCarlo(myEvent)
myAlgoMonteCarlo.setMaximumOuterSampling(maximumOuterSampling)
myAlgoMonteCarlo.setBlockSize(blockSize)
myAlgoMonteCarlo.setMaximumCoefficientOfVariation(coefficientOfVariation)

# Define the HistoryStrategy to store the values of the probability estimator
# and the variance estimator
# used ot draw the convergence graph
# Full strategy
myAlgoMonteCarlo.setConvergenceStrategy(Full())

# Perform the simulation
myAlgoMonteCarlo.run()

# Display number of iterations and number of evaluations
# of the limit state function
print ("Number of evaluations of the limit state function = ", myAlgoMonteCarlo.getResult().getOuterSampling()* myAlgoMonteCarlo.getResult().getBlockSize())

# Display the Monte Carlo probability of 'myEvent'
print ("Monte Carlo probability estimation = ", myAlgoMonteCarlo.getResult().getProbabilityEstimate())

# Display the variance of the Monte Carlo probability estimator
print ("Variance of the Monte Carlo probability estimator = ", myAlgoMonteCarlo.getResult().getVarianceEstimate())

# Display the confidence interval length centered around
# the MonteCarlo probability MCProb
# IC = [MCProb - 0.5*length, MCProb + 0.5*length]
# level 0.95

print ("0.95 Confidence Interval = [", myAlgoMonteCarlo.getResult().getProbabilityEstimate() - 0.5*myAlgoMonteCarlo.getResult().getConfidenceLength(0.95), ", ", myAlgoMonteCarlo.getResult().getProbabilityEstimate() + 0.5*myAlgoMonteCarlo.getResult().getConfidenceLength(0.95), "]")
#print ("")

# Draw the convergence graph and the confidence intervalle of level alpha
alpha = 0.90
convergenceGraphMonteCarlo = myAlgoMonteCarlo.drawProbabilityConvergence(alpha)
# In order to see the graph whithout creating the associated files
View(convergenceGraphMonteCarlo).show()

# Create the file .EPS
#convergenceGraphMonteCarlo.draw("convergenceGrapheMonteCarlo",640,480)
View(convergenceGraphMonteCarlo).show()

############
Monte Carlo
############

Number of evaluations of the limit state function =  16800
Monte Carlo probability estimation =  0.006011904761904769
Variance of the Monte Carlo probability estimator =  3.592614410933927e-07
0.95 Confidence Interval = [ 0.004837133281222889 ,  0.007186676242586648 ]


C:\Users\Marzieh\Anaconda3\lib\site-packages\matplotlib\figure.py:397: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
